In [1]:
import pandas as pd
from chemprop.data import utils
from random import randint
import pandas as pd
import numpy as np
import warnings; warnings.simplefilter('ignore')

In [6]:
def get_partition_as_df(partition):
    dict = {'ID': partition.compound_names(), 'SMILES': partition.smiles()}
    df = pd.DataFrame(dict)
    df2 = pd.DataFrame(df)
    #df2[target_names] = pd.DataFrame(df2.targets.values.tolist(), index= df2.index)
    #df2.drop('targets', axis=1, inplace=True)
    return df2


def random_split(dataset, seed_val):
    # get target names (assuming that 1st column contains molecule names and 2nd column contains smiles and rest of the columns are targets)
    df = pd.read_csv(dataset, sep=",", index_col=None, dtype={'ID': str})
    cols = list(df.columns)
    #target_names = cols[2:]

    mol_dataset = utils.get_data(dataset, use_compound_names=True)
    train, valid, test = utils.split_data(mol_dataset, sizes=(0.7, 0.15, 0.15), seed=seed_val)
    train_df = get_partition_as_df(train)
    train_df = train_df[['ID']]
    valid_df = get_partition_as_df(valid)
    valid_df = valid_df[['ID']]
    test_df = get_partition_as_df(test)
    test_df = test_df[['ID']]
    return train_df, valid_df, test_df

In [7]:
########################################################
# example usage to random split the dataset 5 times

dataset = r'C:\Users\meryc\Dropbox\Mestrado\Multitask\scripts\Birds Toxicity\data_classification\Anas_platyrhynchos_class.csv'

out_dir = '../data/random_split/'

print('started splitting!')

for k in range(5):
    print('fold: %s' % k)
    seed_val = randint(1, 50000)
    print('using seed: %s' % seed_val)
    train, test, valid = random_split(dataset, seed_val) # change split type and see value here
    train_file_path = f"{out_dir}train_fold_{str(k)}.csv"
    valid_file_path = f"{out_dir}valid_fold_{str(k)}.csv"
    test_file_path = f"{out_dir}test_fold_{str(k)}.csv"
    train.to_csv(train_file_path, index=None)
    test.to_csv(test_file_path, index=None)
    valid.to_csv(valid_file_path, index=None)
    print(train.shape)
    print(test.shape)
    print(valid.shape)

print('finished splitting!')

started splitting!
fold: 0
using seed: 43685




  0%|                                                                                          | 0/568 [00:00<?, ?it/s]

  6%|████▌                                                                           | 32/568 [00:00<00:01, 317.41it/s]

 39%|██████████████████████████████▌                                               | 223/568 [00:00<00:00, 1250.86it/s]

  0%|                                                                                          | 0/568 [50:49<?, ?it/s]


100%|███████████████████████████████████████████████████████████████████████████████| 568/568 [00:00<00:00, 894.25it/s]


(397, 1)
(85, 1)
(86, 1)
fold: 1
using seed: 15338


100%|██████████████████████████████████████████████████████████████████████████████| 568/568 [00:00<00:00, 3101.92it/s]


(397, 1)
(85, 1)
(86, 1)
fold: 2
using seed: 15873


...

(397, 1)
(85, 1)
(86, 1)
fold: 3
using seed: 30836


100%|██████████████████████████████████████████████████████████████████████████████| 568/568 [00:00<00:00, 2579.30it/s]


(397, 1)
(85, 1)
(86, 1)
fold: 4
using seed: 15945


100%|██████████████████████████████████████████████████████████████████████████████| 568/568 [00:00<00:00, 3268.50it/s]

(397, 1)
(85, 1)
(86, 1)
finished splitting!


In [8]:
# read dataset and join features (random split)
data = pd.read_csv(r'C:\Users\meryc\Dropbox\Mestrado\Multitask\scripts\Birds Toxicity\data_classification\Anas_platyrhynchos_class.csv', sep=',', dtype={'ID': str})

# join features
data_df=data

# read training and test ids for each fold
for i in range(5):
    train = f"../data/random_split/train_fold_{i}.csv"
    test = f"../data/random_split/test_fold_{i}.csv"
    valid = f"../data/random_split/valid_fold_{i}.csv"

    train_df = pd.read_csv(train, dtype={'ID': str})
    test_df = pd.read_csv(test, dtype={'ID': str})
    valid_df = pd.read_csv(valid, dtype={'ID': str})
    
    # join with data
    train_df = pd.merge(train_df, data_df, on='ID')
    test_df = pd.merge(test_df, data_df, on='ID')
    valid_df = pd.merge(valid_df, data_df, on='ID')
    
    train_df.to_csv(f"../data/random_split/train_fold_{i}.csv", index=False)
    test_df.to_csv(f"../data/random_split/test_fold_{i}.csv", index=False)
    valid_df.to_csv(f"../data/random_split/valid_fold_{i}.csv", index=False)